In [1]:
import math
from typing import Sequence, List
import pandas as pd

# ---------- helpers ----------

def interp_linear(x_tab: Sequence[float], y_tab: Sequence[float], x: float) -> float:
    """Linear interpolate y(x) on ascending x_tab; clamp at ends."""
    n = len(x_tab)
    if n == 0:
        return float("nan")
    if x <= x_tab[0]:
        return float(y_tab[0])
    if x >= x_tab[-1]:
        return float(y_tab[-1])
    lo, hi = 0, n - 1
    while hi - lo > 1:
        mid = (lo + hi) // 2
        if x_tab[mid] <= x:
            lo = mid
        else:
            hi = mid
    x0, x1 = x_tab[lo], x_tab[hi]
    y0, y1 = y_tab[lo], y_tab[hi]
    if x1 == x0:
        return float(y0)
    w = (x - x0) / (x1 - x0)
    return float(y0 + w * (y1 - y0))

def shift_rating_axis(stage_axis: Sequence[float], shift_val: float) -> List[float]:
    """Apply time-based shift to the *rating curve axis* (not to stage): H' = H - s(t)."""
    return [float(h) - float(shift_val) for h in stage_axis]

def blank_result() -> dict:
    """Return blanks (NaNs) for all derived fields."""
    return {
        "QR_base_cfs": math.nan,
        "Fr_ft": math.nan,
        "Fm_ft": math.nan,
        "ratio_Fm_over_Fr": math.nan,
        "factor": math.nan,
        "Q_new_cfs": math.nan,
    }

# ---------- one-step forward model (stage -> discharge) ----------

def discharge_from_stage_at_time(
    GH_thebes_t: float,          # unshifted Thebes GH at time t
    GH_cape_t: float,            # Cape GH at time t
    shift_t: float,              # time-based shift at Thebes for time t
    rating_stage_axis: Sequence[float],
    rating_q_axis: Sequence[float],
    fall_stage_axis: Sequence[float],
    fall_fr_axis: Sequence[float],
    factor_ratio_axis: Sequence[float],
    factor_val_axis: Sequence[float],
    Ccape: float,
    Cthebes: float
) -> dict:

    # Require all three inputs present and finite
    if not (pd.notna(GH_thebes_t) and pd.notna(GH_cape_t) and pd.notna(shift_t)):
        return blank_result()
    if not (math.isfinite(float(GH_thebes_t)) and math.isfinite(float(GH_cape_t)) and math.isfinite(float(shift_t))):
        return blank_result()

    # 1) Temporarily shifted rating curve for this time step; evaluate at raw GH
    rating_axis_t = shift_rating_axis(rating_stage_axis, float(shift_t))
    QR = interp_linear(rating_axis_t, rating_q_axis, float(GH_thebes_t))

    # 2) Rated fall uses *unshifted* GH
    Fr = interp_linear(fall_stage_axis, fall_fr_axis, float(GH_thebes_t))

    # 3) Measured fall uses pure elevations (no shift)
    Fm = (float(GH_cape_t) + float(Ccape)) - (float(GH_thebes_t) + float(Cthebes))

    # 4) Ratio and factor
    if not math.isfinite(Fr) or Fr <= 0.0:
        return {
            "QR_base_cfs": QR,
            "Fr_ft": Fr,
            "Fm_ft": Fm,
            "ratio_Fm_over_Fr": math.nan,
            "factor": math.nan,
            "Q_new_cfs": math.nan,
        }

    r = Fm / Fr
    fac = interp_linear(factor_ratio_axis, factor_val_axis, r)

    # 5) Backwater-adjusted discharge
    Q_new = QR * fac if math.isfinite(QR) and math.isfinite(fac) else math.nan

    return {
        "QR_base_cfs": QR,
        "Fr_ft": Fr,
        "Fm_ft": Fm,
        "ratio_Fm_over_Fr": r,
        "factor": fac,
        "Q_new_cfs": Q_new,
    }

def main():
    # Load tables
    rating_df = pd.read_csv("rating_table.csv")
    fall_df   = pd.read_csv("fall_rated.csv")
    factor_df = pd.read_csv("factor_table.csv")

    # Choose stage-axis column name for rating table
    stage_col = "GsH_ft" if "GsH_ft" in rating_df.columns else ("GH_ft" if "GH_ft" in rating_df.columns else None)
    if stage_col is None:
        raise ValueError("rating_table.csv must have a stage column named 'GsH_ft' or 'GH_ft'.")

    # Sort to ensure ascending axes
    rating_df = rating_df.sort_values(stage_col)
    fall_df   = fall_df.sort_values("GH_ft")
    factor_df = factor_df.sort_values("ratio")

    rating_stage_axis = rating_df[stage_col].astype(float).to_list()
    rating_q_axis     = rating_df["Q_cfs"].astype(float).to_list()
    fall_stage_axis   = fall_df["GH_ft"].astype(float).to_list()
    fall_fr_axis      = fall_df["Fr_ft"].astype(float).to_list()
    factor_ratio_axis = factor_df["ratio"].astype(float).to_list()
    factor_val_axis   = factor_df["factor"].astype(float).to_list()

    # Load time series
    thebes = pd.read_csv("thebes_stage.csv", parse_dates=["time"]).sort_values("time")
    cape   = pd.read_csv("cape_stage.csv",   parse_dates=["time"]).sort_values("time")
    shift  = pd.read_csv("shift_series.csv", parse_dates=["time"]).sort_values("time")

    # Align by time (do NOT fill missing shift—per your requirement)
    df = thebes.merge(cape, on="time", how="outer") \
               .merge(shift, on="time", how="outer") \
               .sort_values("time")

    # Datum constants (same as your workbook)
    Ccape   = 304.27
    Cthebes = 299.70

    # Compute discharge row-by-row ONLY when all three inputs exist
    results = []
    for row in df.itertuples(index=False):
        vals = discharge_from_stage_at_time(
            GH_thebes_t       = getattr(row, "GH_Thebes_ft", float("nan")),
            GH_cape_t         = getattr(row, "Cape_GH_ft", float("nan")),
            shift_t           = getattr(row, "shift_ft", float("nan")),
            rating_stage_axis = rating_stage_axis,
            rating_q_axis     = rating_q_axis,
            fall_stage_axis   = fall_stage_axis,
            fall_fr_axis      = fall_fr_axis,
            factor_ratio_axis = factor_ratio_axis,
            factor_val_axis   = factor_val_axis,
            Ccape             = Ccape,
            Cthebes           = Cthebes
        )
        results.append(vals)

    out = pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)

    # Save (NaNs will appear as blank cells in CSV)
    out.to_excel("q_from_usgs_stage.xlsx", index=False)
    print(f"Wrote q_from_usgs_stage.xlsx with {len(out)} rows")

if __name__ == "__main__":
    main()


Wrote q_from_usgs_stage.xlsx with 129288 rows
